In [160]:
# Install necessary packages

In [161]:
import numpy as np
import pandas as pd

In [162]:
# Import origional CSV file for all inspections

In [163]:
inspections = pd.read_csv('inspections.csv')

In [164]:
# Create function to translate to snake case
# Import regular expressions, practiced using regex101.com
import re

In [180]:
def snake_case(title):
    title = title.lower().replace(" ", "_")
    title = re.sub('\W+',"", title)
    if len(title) > 1:
        if title[-1] == "_":
            return title[:-1]
    return title

In [178]:
# Snake case funtion test

In [181]:
string = "HEllo you PErson!@"


snake_case(string)

'hello_you_person'

In [182]:
inspections.columns = [snake_case(x) for x in inspections.columns]

In [184]:
inspections

,inspection_id,dba_name,aka_name,license,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2166551,DOLLOP COFFEE CO,DOLLOP COFFEE CO,2488257.0,Restaurant,Risk 2 (Medium),5500 S UNIVERSITY,CHICAGO,IL,60637.0,04/23/2018,Complaint,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.795008,-87.598280,"(41.795007727516236, -87.59827950081906)"
1,2166532,UNIVERSITY OF CHICAGO BARTLETT COMMONS,UNIVERSITY OF CHICAGO BARTLETT COMMONS,1197090.0,Restaurant,Risk 1 (High),5640 S UNIVERSITY,CHICAGO,IL,60637.0,04/23/2018,Canvass Re-Inspection,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.792025,-87.598236,"(41.792025410968655, -87.59823635212413)"
2,2166531,MCDONALD,MCDONALD,2428105.0,Restaurant,Risk 2 (Medium),1657 W 95TH ST,CHICAGO,IL,60643.0,04/23/2018,Short Form Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.721073,-87.664898,"(41.721073142167896, -87.66489837339608)"
3,2166533,Josephine Locke Elementary School,Josephine Locke Elementary School,24231.0,School,Risk 1 (High),2828 N Oak Park AVE,CHICAGO,IL,60634.0,04/23/2018,Canvass,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.931763,-87.795641,"(41.931763005765575, -87.79564085495707)"
4,2166526,Randolph Magnet,Randolph Magnet,29111.0,School,Risk 1 (High),7316 S Hoyne (2100W),CHICAGO,IL,60636.0,04/23/2018,Canvass,Pass w/ Conditions,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.760597,-87.676002,"(41.76059741181121, -87.67600175472701)"
5,2166496,LOS ALTOS,LOS ALTOS,2590216.0,Restaurant,Risk 1 (High),1848 W 47TH ST,CHICAGO,IL,60609.0,04/23/2018,License,Pass,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.808647,-87.671778,"(41.80864676132468, -87.6717775148354)"
6,2166491,LOS ALTOS,LOS ALTOS,24099.0,Restaurant,Risk 1 (High),1848 W 47TH ST,CHICAGO,IL,60609.0,04/23/2018,Canvass,Out of Business,NaN,41.808647,-87.671778,"(41.80864676132468, -87.6717775148354)"
7,2166524,GSB CAFE,GSB CAFE,1519437.0,Restaurant,Risk 1 (High),5807 S WOODLAWN AVE,CHICAGO,IL,60637.0,04/23/2018,Complaint Re-Inspection,Pass,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,41.789364,-87.596263,"(41.78936441141914, -87.59626269511155)"
8,2166511,666 FOOD MART INC.,666 FOOD MART INC.,2584050.0,Grocery Store,Risk 2 (Medium),1825 W 59TH ST,CHICAGO,IL,60636.0,04/23/2018,License,Fail,38. VENTILATION: ROOMS AND EQUIPMENT VENTED AS...,41.786564,-87.670239,"(41.786564006417, -87.67023898101154)"
9,2166510,"NOOR FOOD MARKET, INC.",NOOR FOOD MARKET,2354287.0,Grocery Store,Risk 2 (Medium),1825 W 59TH ST,CHICAGO,IL,60636.0,04/23/2018,Canvass,Out of Business,NaN,41.786564,-87.670239,"(41.786564006417, -87.67023898101154)"
